<a href="https://colab.research.google.com/github/alizehadeel/AI-ML-Notebook/blob/main/MultiLabelClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dataset

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alizehadeel","key":"a967484beac04826fa3b66060c694b64"}'}

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/


In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d meherunnesashraboni/multi-label-image-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/meherunnesashraboni/multi-label-image-classification-dataset
License(s): ODC Public Domain Dedication and Licence (PDDL)
 89% 655M/738M [00:03<00:01, 54.6MB/s]
100% 738M/738M [00:03<00:00, 202MB/s] 


In [6]:
!unzip multi-label-image-classification-dataset.zip -d multi_label_dataset

Streaming output truncated to the last 5000 lines.
  inflating: multi_label_dataset/multilabel_modified/images/image3561.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3562.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3563.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3564.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3565.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3566.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3567.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3568.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3569.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image357.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3570.jpg  
  inflating: multi_label_dataset/multilabel_modified/images/image3571.jpg  
  inflating: multi_label_dataset/multi

# Connecting to drive

In [7]:
from google.colab import drive
drive.mount('/content/drive') #Create a folder named drive inside /content, and link it (mount it) to my Google Drive account

Mounted at /content/drive


In [9]:
#/content/drive/MyDrive/  Inside that mounted folder, Colab mirrors your actual Google Drive structure:
#This MyDrive folder is literally your main Google Drive (the same one you see on drive.google.com).
#So if you had a folder in Drive like: MyDrive/kaggle_datasets/data.csv
#You can now access it in Colab as: '/content/drive/MyDrive/kaggle_datasets/data.csv'

In [11]:
!ls /content/drive/MyDrive #this is mirroring my drive


 11b.asm
'1a. Intro to DS - Lecture 1.gdoc'
'1b. Lecture 2 Time Complexity.gdoc'
'23.4 - Lab Manual - Polymorphism.gdoc'
'23L0674 (1).gdoc'
 23L0674.gdoc
 23L0674Q1.cpp.gdoc
'25.4 - Lab Manual -Templates.gdoc'
'2. Lists 1.gdoc'
'A1-8_oct-evaluation (1).gsheet'
'A1-8_oct-evaluation (2).gsheet'
 A1-8_oct-evaluation.gsheet
 A2-25_nov-evaluation.gsheet
 A3-7_Dec-evaluation.gsheet
 Aaiza.jpg
'AI ML Notebook.gdoc'
'Alizeh Adeel Resume.gdoc'
'Assignment 1 DS (Lists and Stack).gdoc'
'Assignment 2-2024.gdoc'
'Blue Dark Professional Geometric Business Project Presentation .gslides'
 Chapter_3.gslides
 CHOPCHOP.gdoc
 CiscoQP.gdoc
 Classroom
 CN-Ch1.gslides
 CN-Ch2.gslides
'Colab Notebooks'
'Computer Networking_ A Top-Down Approach, Global Edition, 8th Edition (1).gdoc'
'Computer Networking_ A Top-Down Approach, Global Edition, 8th Edition.gdoc'
'Copy of 8.4 - Bit Manipulation - Lab Manual - Practice Problem.docx'
'Counting Techniques (1).gdoc'
'Counting Techniques.gdoc'
 CoverLetter.gdoc
'CS Cour

In [12]:
!ls

drive	     multi_label_dataset			   sample_data
kaggle.json  multi-label-image-classification-dataset.zip


In [13]:
!ls multi_label_dataset

multilabel_modified


In [14]:
!ls multi_label_dataset/multilabel_modified/

 images
'multilabel_classification(2).csv'
'multilabel_classification(6)-reduced_modified.csv'
'multilabel_classification(7).csv'
 multilabel_classification.csv


In [18]:
#to see which of the csv to use
import pandas as pd

# View first few rows of each CSV
for name in [
    "multilabel_classification.csv",
    "multilabel_classification(2).csv",
    "multilabel_classification(6)-reduced_modified.csv",
    "multilabel_classification(7).csv"
]:
    print(f"--- {name} ---")
    path = f"/content/multi_label_dataset/multilabel_modified/{name}"
    df = pd.read_csv(path)
    print(df.head(), "\n")


--- multilabel_classification.csv ---
      Column1       Column2
0  Image_Name       Classes
1  image0.jpg              
2  image1.jpg    bus person
3  image2.jpg         sitar
4  image3.jpg        flutes 

--- multilabel_classification(2).csv ---
   Image_Name  \
0  image1.jpg   
1  image2.jpg   
2  image3.jpg   
3  image4.jpg   
4  image5.jpg   

   Classes(motorcycle, truck, boat, bus, cycle, person, desert, mountains, sea, sunset, trees, sitar, ektara, flutes, tabla, harmonium)  \
0                                         bus person                                                                                      
1                                              sitar                                                                                      
2                                             flutes                                                                                      
3                                          bus trees                                        

# Preprocessing Data

In [22]:
import os
import pandas as pd

base_path='/content/multi_label_dataset/multilabel_modified'
images_path=base_path+'/images'
labels_path=base_path+ '/multilabel_classification(2).csv'

df=pd.read_csv(labels_path)
df.head()


,Image_Name,"Classes(motorcycle, truck, boat, bus, cycle, person, desert, mountains, sea, sunset, trees, sitar, ektara, flutes, tabla, harmonium)",motorcycle,truck,boat,bus,cycle,person,desert,mountains,sea,sunset,trees,sitar,ektara,flutes,tabla,harmonium
0,image1.jpg,bus person,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,image2.jpg,sitar,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,image3.jpg,flutes,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,image4.jpg,bus trees,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,image5.jpg,bus,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
from sklearn.model_selection import train_test_split

train_df, temp_df=train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df=train_test_split(temp_df, test_size=0.5, random_state=42)

print("Train shape:" + str(train_df.shape))
print("Validation shape:" + str(val_df.shape))
print("Test shape:" + str(test_df.shape))

Train shape:(6277, 18)
Validation shape:(1345, 18)
Test shape:(1346, 18)


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255)#multiply each pixel with 1/255 to rescale
img_size=[128, 128]


In [ ]:
#this is where your data (images + labels) finally get connected and prepared for training.
train_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=images_path,
    x_col='Image_Name',
    y_col=list(train_df.columns[2:]),
    target_size=img_size,
    class_mode='raw'
    batch_size=32,
    shuffle=True
)